<a href="https://colab.research.google.com/github/vinesh-245/AI-evaluation/blob/main/ik_agenteval_llm_judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent Evaluation Homework for non-SWE

This module demonstrates using LangSmith to monitor your agent in production. You need to have created a langsmith account, and have your custom openAI keys.


Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.

Note that observability is important throughout all stages of application development - from prototyping, to beta testing, to production. There are different considerations at all stages, but they are all intricately tied together. In this tutorial we walk through the natural progression.

Let's assume that we're building a simple RAG application using the OpenAI SDK. The simple application we're adding observability to looks like.

Link to homework document: https://docs.google.com/document/d/1IrgnVLXtHcx0TJ0IJlUzrHsqseM23eGh7DdpYEkzJvU/edit?tab=t.0


In [ ]:
import os
!pip install langsmith
from openai import OpenAI
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "default"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "default"
#os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
#os.environ["LANGSMITH_API_KEY"] =  userdata.get('LANGSMITH_API_KEY')
os.environ["OPENAI_API_KEY"] = "<enter-your-own-key-here>"
os.environ["LANGSMITH_API_KEY"] = "<enter-your-own-key-here>"

# **Part 1 - Running the first Basic App**

Let's first build a basic LLM application, with a basic calls wrapped around LangSmith. Run the below, and check out what is showing up on the LangSmith Portal.

In [ ]:
openai_client = wrap_openai(OpenAI())

def retriever(query: str):
    results = ["Harrison worked at Kensho"]
    return results

def rag(question):
    docs = retriever(question)
    system_message = """Answer the users question using only the provided information below:

    {docs}""".format(docs="\n".join(docs))

    response = openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ],
        model="gpt-4o-mini",
    )
    return response.choices[0].message.content

# **Part 2 - Run the LangSmith enhanced version**

The first thing you might want to trace is all your OpenAI calls. After all, this is where the LLM is actually being called, so it is the most important part! We've tried to make this as easy as possible with LangSmith by introducing a dead-simple OpenAI wrapper. All you have to do is modify your code to look something like:

Notice how we import from langsmith.wrappers import wrap_openai and use it to wrap the OpenAI client (openai_client = wrap_openai(OpenAI())).


Notice how we import from langsmith import traceable and use it decorate the overall function (@traceable).


Notice how we import from langsmith import traceable and use it decorate the overall function (@traceable(run_type="retriever")).

What happens if you call it in the following way?

# Beta Testing
The next stage of LLM application development is beta testing your application. This is when you release it to a few initial users. Having good observability set up here is crucial as often you don't know exactly how users will actually use your application, so this allows you get insights into how they do so. This also means that you probably want to make some changes to your tracing set up to better allow for that. This extends the observability you set up in the previous section

# Collecting Feedback
A huge part of having good observability during beta testing is collecting feedback. What feedback you collect is often application specific - but at the very least a simple thumbs up/down is a good start. After logging that feedback, you need to be able to easily associate it with the run that caused that. Luckily LangSmith makes it easy to do that.

First, you need to log the feedback from your app. An easy way to do this is to keep track of a run ID for each run, and then use that to log feedback. Keeping track of the run ID would look something like:

# Logging Metadata
It is also a good idea to start logging metadata. This allows you to start keep track of different attributes of your app. This is important in allowing you to know what version or variant of your app was used to produce a given result.

For this example, we will log the LLM used. Oftentimes you may be experimenting with different LLMs, so having that information as metadata can be useful for filtering. In order to do that, we can add it as such:

In [ ]:
openai_client = wrap_openai(OpenAI())

@traceable(run_type="retriever")
def retriever(query: str):
    results = ["Harrison worked at Kensho"]
    return results


@traceable(metadata={"llm": "gpt-4o-mini"})
def rag(question):
    docs = retriever(question)
    system_message = """Answer the users question using only the provided information below:

    {docs}""".format(docs="\n".join(docs))

    response = openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ],
        model="gpt-4o-mini",
    )

    return response.choices[0].message.content


# Let's do a successful run
import uuid

run_id = str(uuid.uuid4())
rag(
    "where did harrison work",
    langsmith_extra={"run_id": run_id, "metadata": {"user_id": "harrison"}}
)

from langsmith import Client
ls_client = Client()
ls_client.create_feedback(
    run_id,
    key="user-score",
    score=1.0,
)

# Let's do a bad  run
run_id = str(uuid.uuid4())
print(run_id)

rag(
    "where did peter work",
    langsmith_extra={"run_id": run_id, "metadata": {"user_id": "peter"}}
)

ls_client.create_feedback(
    run_id,
    key="user-score",
    score=0.0,
)

Observing what happens

You have now just run a app with Langsmith integreated with 1)tracing, 2)extra metadata, 3)feedback.

After successfully running this, let's goto the LangSmith website to check out
1)the traces from Rag to retriever to the OpenAI call
2)the metadata that we added on user_id and llm model
3)the feedback added for the track.


Next, let's try to query by feedback that is positive

Next, let's try to query the feedback that is negative.

Last, let's look at the monitoring dashboard.